In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
from selenium import webdriver
from selenium.common import exceptions
import re
import tensorflow as tf
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# 크롤링

In [3]:
OUTPUT_FILE_NAME = 'output.txt'

In [4]:
wd = "C:\\Users\\user\\Downloads\\chromedriver_win32\\chromedriver.exe"

In [5]:
import urllib.request
def get_text(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
    text = ''
    for item in soup.find_all('div', id='realArtcContents'):
        text = text + str(item.find_all(text=True))
    return text

In [6]:
def clean_text(text):
    cleaned_text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\.]+", " ", str(text))
    return cleaned_text

In [7]:
def get_title(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'html.parser', from_encoding='utf-8')
    text = soup.find('h3',class_='articleSubecjt')
    return text

In [8]:
import time

def comment(url):
    driver = webdriver.Chrome(wd)
    url_nums = re.findall("\d+", url)
    addr = 'http://comm.news.nate.com/Comment/ArticleComment/List?artc_sq='+url_nums[0]+'n'+url_nums[1] # 댓글창 주소
    driver.get(addr)
    pages = 2
    try:
        for i in range(5):
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"body > div.commentBox.reply_hide > div:nth-child(3) > div.paging_wrap > a:nth-child("+str(pages)+")"))).click()
            time.sleep(2)
            print(pages, end=" ")
            pages+=1
    except exceptions.ElementNotVisibleException as e: # 페이지 끝
        pass
    except Exception as e: # 다른 예외 발생시 확인
        print(e)
    html = driver.page_source
    dom = BeautifulSoup(html, "lxml")
    # 댓글이 들어있는 페이지 전체 크롤링
    comments_raw = dom.find_all('dd',{'class':'usertxt'})
    # 댓글의 text만 뽑는다.
    comments = [comment.text.strip() for comment in comments_raw]
    return comments

# 키워드 추출

In [9]:
from krwordrank.sentence import summarize_with_sentences
def keyword_word(text):#키워드 단어 추출
    data = text.split('.')
    keywords, sents = summarize_with_sentences(data, num_keywords=10, num_keysents=10)
    return keywords

In [10]:
def keyword_sentence(url):#키워드 문장 추출
    m_text = get_text(url)
    r_text = clean_text(m_text)
    data = r_text.split('.')
    keywords, sents = summarize_with_sentences(data, num_keywords=10, num_keysents=10)
    return sents

# 모델 로딩

In [11]:
from tensorflow.keras.models import load_model
from konlpy.tag import Okt  
okt = Okt()

In [12]:
def model_load(comments):
    token_sent=[]
    for comment in comments:
        temp = okt.morphs(comment, stem=True)
        token_sent.append(temp)
    max_words = 35000
    tokenizer = Tokenizer(num_words = max_words) # 상위 35,000개의 단어만 보존
    tokenizer.fit_on_texts(token_sent) 
    token_sent = tokenizer.texts_to_sequences(token_sent)
    word_to_index = tokenizer.word_index
    vocab_size = len(word_to_index)+1
    max_len = 124
    X_data = pad_sequences(token_sent, maxlen=max_len)
    model = load_model('model8n.h5')
    predict = model.predict_classes(X_data)
    a = [1 for _ in range(len(X_data))]
    for i in range(len(X_data)):
        if(predict[i] == 0):
            a[i]=0
    return a

# 댓글 수치 판단

In [13]:
def neg_pos(results):
    pos = 0
    neg = 0
    for result in results:
        if result == 0:
            pos += 1
        else:
            neg += 1
    result = pos / (pos + neg) * 100
    return result

In [14]:
def clean(results):
    for result in results:
        if result == 1:
            result == 'bad comment'
    return result

# 악플 개수 세기

In [15]:
def count(num): #count는 악플 개수, number은 악플 위치
    count = 0
    number = []
    for i in range(1, len(num)):
        if num[i] == 1:
            count += 1
            number.append(i)
    return count, number

# 악플 데이터 삭제

In [16]:
def de_comment(comments, number):
    new = np.delete(comments, number)
    return new

# 악플 데이터 남기기

In [17]:
def count2(num): #count는 악플 개수, number은 악플 위치
    number = []
    for i in range(1, len(num)):
        if num[i] == 0:
            number.append(i)
    return number

In [18]:
def de_comment2(comments, number):
    new = np.delete(comments, number)
    return new

# 선플 데이터 넣기

In [19]:
actor = ["요즘 연기 정말 잘보고있어요! 항상 응원해요",
        "항상 열심히 하는 모습 너무 보기 좋아요:)",
        "최근 작품에서 연기 정말 많이 늘었더라고요!",
        "연기가 너무 좋아서 응원해요~",
        "요즘 내가 가장 좋아하는 배우 중 한 명ㅎㅎ",
        "어떤 연기를 해도 보는 사람의 심금을 울리는 배우",
        "요즘 얼굴 정말 좋아보여요",
        "예전부터 지금까지 계속해서 응원하고 있어요:)",
        "연기로는 절대 뭐라 못하는 배우",
        "연기신",
        "정말 얼굴 천재..그 자체",
        "요즘 너무 좋아요!",
        "진짜 송강호급 연기자다 연기레알 개잘함.",
        "연기 너무 잘해ㅠㅠㅠㅠ",
        "이번년도 연기 대상 주자!!!!!!",
        "발성.발음.표정...참 몰입되는 배우님...응원합니다.",
        "대상 받아야해요....",
        "난 옛날부터연기잘하는거알고있었지.. ",
        "사랑해요!!!!!!!!!!!",
        "걍 지린다.. 연기 너무 잘하시는 듯",
        "진짜 연기 미쳤어.......",
        "진짜 깔게 없는 연기력 .. ",
        "표정 연기가 너무 섬세하셔서 숨 안쉬고 보게 되는.. 응원합니다 배우님 :>",
        "연기 볼 때마다 놀라요 흡입력이 진짜.. ",
        "진심 시상식에서 상 받아야할 신들린 연기자"]

In [20]:
singer = ["미세먼지 한톨도 안먹은 목소리의 주인공 진짜.......",
         "음악성은 진짜..말도 안된다고 생각한다.. ",
         "한국 가수 역사상 최고인 사람",
         "목소리가 진짜 보물이야",
         "목소리가 보석같다",
         "외로움과 따뜻함을 모두 가진 가수 목소리를 듣고 있으면 내면의 외로움이 느껴지고 그게 안쓰러워 다가가면 그가 가진 따스함에 오히려 위로 받게 되는 나 자신을 발견하게 됨..",
         "같은 시대에 태어나 이 목소리를 들을 수 있다는건 정말 해운이야",
         "음색도 너무 좋지만 말하는거 보면 말도 따뜻하고 예쁘게 하는 듯",
         "어떻게 사람이 이렇게 모든게 완벽하냐",
         "이 사람의 노래를 들으며 내 인생의 슬픔을 치유받았다..",
         "나의 사랑하는 아티스트",
         "내 인생 가수",
         "정말 몇 번을 들어도 소름돋는 가창력 음색",
         "음색도 뭐도 다 좋지만 감성이 너무 좋다",
         "먼 미래에 더 더 빛나는 뮤지션으로 재평가 될 가수",
         "항상 이 목소리 유지해줘 ㅠㅠ 항상 응원해",
         "당신을 보고 눈물이 나는건 당신을 위로하고 싶은 나 때문인지 당신한테 위로받고 싶은 나 때문인지",
         "정말 강한 사람 같다. 내가 자식을 낳는다면 이렇게 키우고 싶어",
         "감성, 보컬실력, 섬세한 삼박자 모두 갖춘 최고의가수",
         "콘서트 한 번만 가보면 소원이 없겠다 진짜로",
         "독보적인 가수",
         "음원보다 라이브가 훠어어얼씬 좋은 가수",
         "진짜 사기캐..사람이 어떻게 이렇게 완벽하냐",
         "새삼스럽게 말하지만 노래 진짜 잘해",
         "언제나 응원합니다:)"]

In [21]:
entertain = ["강약 조절이 능숙한 개그맨..진짜 어디서도 못구함",
            "진짜 떡잎부터 도른자 ㅋㅋㅋㅋ너무 웃겨",
            "코미디언 중에서 탑이다 ㅋㅋ",
            "남녀 통틀어서 이런사람 드물거같아 ㅋㅋ",
            "정말 박미선처럼 롱런하면 좋겠다",
            "입담이나 재치가 진짜 너무 사기캐..",
            "막 힘들어간 억지 개그 없이 말하나 행동 하나하나 너무 웃겨서 좋아요 ㅋㅋㅋㅋ",
            "개좋아 진짜로 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ",
            "내 호적메이트면 좋겠다 너무 좋음ㅋㅋㅋㅋ",
            "개그가 기분 나쁘지 않고 깔끔함",
            "이번년도에 대상 받자!!!!!",
            "진짜 너무 재미있어 ㅋㅋㅋㅋ",
            "진짜 장난 아니게 웃기고 센스 쩔고 사랑스러운데 올해 상 휩쓸고 다니자",
            "찜찜하고 강제로 웃게 만드는게 아니라 진짜 자연스럽고 유머스러움에 감탄하며 건강한 웃음을 재창조해낸 이 시대 최고",
            "진짜 말하는 것부터 하는 액션까지 전부 따 뼛속부터 개그맨 ㅋㅋ",
            "성격 진짜 좋아보이더라",
            "말빨로 웃기는 연옌 드문데 진짜 드문케이스.",
            "센스있게 웃기기 쉽지 않은데 진짜 센스가 장난 아니고 성격도 진실되어보임",
            "내 롤모델이다 진짜로 ㅋㅋㅋㅋㅋ",
            "너무 특색있고 오바하지 않는데도 웃겨 ㅋㅋㅋ",
            "진짜 타고나기를 개그맨이다..",
            "개그 진짜 내 취향임",
            "너무 열심히 하고 기본 인성이 착한듯",
            "보는 사람이 기분 하나도 나쁘지 않고 센스있게 웃겨서 좋음 ㅠㅠㅠㅠㅠㅠㅠ",
            "저렇게 웃기고 재치있고 센스있고 귀여운 사람이 세상에 또 어딨어 ㅋㅋ"]

In [22]:
virtue = ['진짜 클라스 보소 역시..',
         "갓갓",
         "이게 뭐라고 눈이 촉촉히 젖어오네요",
         "머리로는 이렇게 해야한다, 이런행동을 해야한다고 생각해도 행동으로 옮기긴 정말 힘들텐데..대단하다",
         "이런 선행이 더 많아지면 좋겠다",
         "인성짱짱이다",
         "진짜 착하다...",
         "존경스럽다",
         "진짜 멋있는거같다 ㅠㅠ",
         "사람이 왜 칭찬받고 하는지 보면 다 이유가 있는데 정말 착하다",
         "당신은 도덕책..",
         "참 멋있는 사람이네요.",
         "이런일 하는게 쉬운 일은 아닐텐데 진짜 멋있다",
         "세상은 아직 살만하다는걸 느낀다",
         "참 대단하네요 마인드가..존경스럽습니다",
         "세상이 더럽다고 느끼고 좌절할 때 쯤 이런 기사를 보게돼서 다행이다",
         "굉장히 교양있어보인다..",
         "크 멋있어라 적게 일하고 돈 많이 벌길~",
         "가정 교육 제대로 받았네",
         "앞으로 좋은 꽃길만 걷길...",
         "아름다워서 눈물이 날려하네요  참 착해요",
         "천사가 여기있네",
         "이런 사람보고 성인이라고 하는거지",
         "보면 볼수록 정말 괜찮은 사람이라는게 느껴진다",
         "연예인이 아니라 인간대 인간으로 보아도 괜찮은 사람같다",
         "크 클라스 보소",
         "참 좋은 사람인 것 같아",
         "사람을 행복하게 하는 비타민 같은 존재..항상 좋은 일만 있으면 좋겠다",
         "인성 진자 최고인거같다",
         "인성킹"]

In [23]:
import string
import random
result = []

def select(text, number):#기사 내용 중 단어 선별하기
    lines = []
    line2 = []
    for word in text:
        lines = str.split(text,".")
    for each_line in lines:
        if each_line.find("기부") > 0:
            result = random.sample(virtue, number)
            break
        elif each_line.find("선행") > 0:
            result = random.sample(virtue, number)
            break
        elif each_line.find("연기") > 0:
            result = random.sample(actor, number)
            break
        elif each_line.find("배우") > 0:
            result = random.sample(actor, number)
            break 
        elif each_line.find("작품") > 0:
            result = random.sample(actor, number)
            break
        elif each_line.find("차기작") > 0:
            result = random.sample(actor, number)
            break
        elif each_line.find("영화") > 0:
            result = random.sample(actor, number)
            break
        elif each_line.find("드라마") > 0:
            result = random.sample(actor, number)
            break
        elif each_line.find("시청") > 0:
            result = random.sample(actor, number)
            break
        elif each_line.find("음색") > 0:
            result = random.sample(singer, number)
            break
        elif each_line.find("콘서트") > 0:
            result = random.sample(singer, number)
            break
        elif each_line.find("무대") > 0:
            result = random.sample(singer, number)
            break
        elif each_line.find("노래") > 0:
            result = random.sample(singer, number)
            break
        elif each_line.find("가수") > 0:
            result = random.sample(singer, number)
            break
        elif each_line.find("그룹") > 0:
            result = random.sample(singer, number)
            break
        elif each_line.find("아이돌") > 0:
            result = random.sample(singer, number)
            break
        elif each_line.find("걸그룹") > 0:
            result = random.sample(singer, number)
            break
        elif each_line.find("춤") > 0:
            result = random.sample(singer, number)
            break
        elif each_line.find("댄스") > 0:
            result = random.sample(singer, number)
            break
        elif each_line.find("예능") > 0:
            result = random.sample(entertain, number)
            break
        elif each_line.find("개그맨") > 0:
            result = random.sample(entertain, number)
            break
        elif each_line.find("개그우먼") > 0:
            result = random.sample(entertain, number)
            break
        elif each_line.find("예능") > 0:
            result = random.sample(entertain, number)
            break
        else:
            break
    return result

# 모델 추가 train

In [24]:
import pandas as pd
from konlpy.tag import Okt
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

def train(comments, model_result):
    comment_data = pd.DataFrame(comments)
    comment_data.columns = ['comment']
    in_result = np.insert(model_result, 0, 0)
    de_result = np.delete(in_result,0)
    de_data = pd.DataFrame(de_result)
    de_data.columns = ['label']
    df_new = pd.concat([de_data, comment_data], axis = 1)
    X_data = df_new['comment']
    y_data = df_new['label']
    normalized_text = []
    for string in X_data.tolist():
        try:
            tokens = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣]+", " ", string.lower())
        except Exception as e:
            break
    normalized_text.append(tokens)
    #df_new.comment = normalized_text
    okt = Okt()
    X_token=[]
    for sentence in df_new['comment']:
        temp_X = []
        temp_X = okt.morphs(sentence, stem=True) # 토큰화
        X_token.append(temp_X)
    max_words = 10000
    tokenizer = Tokenizer(num_words = max_words) # 상위 35,000개의 단어만 보존
    tokenizer.fit_on_texts(X_token) 
    X_token = tokenizer.texts_to_sequences(X_token)
    word_to_index = tokenizer.word_index
    vocab_size = len(word_to_index) + 1
    max_len = 124
    X_data = pad_sequences(X_token, maxlen=max_len)
    y_data = np.array(y_data).reshape(-1, 1)
    model = load_model('model8n.h5')
    history = model.fit(X_data, y_data, epochs=5, batch_size=32)

# 플라스크

In [25]:
from flask import Flask, render_template
from flask import request, send_file, url_for
from wtforms import Form, TextAreaField, validators
import pickle
import sqlite3
import os

In [26]:
app = Flask(__name__)
url = ''

In [27]:
class ArticleForm(Form):
    articleurl = TextAreaField('',[validators.DataRequired()])

In [28]:
@app.route('/')
def index():
    form = ArticleForm(request.form)
    return render_template('article_app.html', form=form)

In [29]:
@app.route('/title', methods = ['POST'])
def article():
    form = ArticleForm(request.form)
    if request.method == 'POST' and form.validate():
        global url
        url = request.form['articleurl']
        title = get_title(url)
        r_title = clean_text(title)
        m_text = get_text(url)
        r_text = clean_text(m_text)
        return render_template('title.html', title=r_title, content=r_text)
    return render_template('article_app.html', form=form)

In [30]:
@app.route('/result')
def review():
    global url
    m_text = get_text(url)
    r_text = clean_text(m_text)
    cr_comment = comment(url)
    results = model_load(cr_comment)
    cm_count, number = count(results)
    new_comment = de_comment(cr_comment, number)
    good_comment = select(r_text, cm_count)
    result1 = neg_pos(results) 
    return render_template('result.html', list1 = new_comment, list2 = result1, list3 = good_comment)

In [31]:
@app.route('/bad_comment')
def bad_comment():
    global url
    cr_comment = comment(url)
    results = model_load(cr_comment)
    number = count2(results)
    new_comment = de_comment2(cr_comment, number)
    return render_template('bad_comment.html', list1 = new_comment)

In [32]:
@app.route('/no_train')
def no_train():
    return render_template('no_train.html')

In [33]:
@app.route('/thanks')
def feedback():
    global url
    cr_comment = comment(url)
    results = model_load(cr_comment)
    train_check = train(cr_comment, results)
    return render_template('thanks.html')

In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Apr/2020 17:00:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2020 17:01:13] "POST /title HTTP/1.1" 200 -


2 3 4 5 6 

127.0.0.1 - - [09/Apr/2020 17:01:41] "GET /result HTTP/1.1" 200 -


2 3 4 5 6 

127.0.0.1 - - [09/Apr/2020 17:02:26] "GET /bad_comment HTTP/1.1" 200 -
